<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Language Models

**Note: For other Non-OpenAI models, you can check out: https://python.langchain.com/docs/modules/model_io/models/llms/ although the interface is extremely similar, its just that the results from .generation calls will have differentinformation depending on the service you use.**

## Text Model Connection

In [1]:
from langchain_openai import OpenAI

You can stored your API key however you prefer, its common to set it as an environment variable, for example:

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'example key'

In [ ]:
print(os.environ['OPENAI_API_KEY'])

Note, that LangChain automatically looks up for any environment variable with the name OPENAI_API_KEY automatically when making a connection to OpenAI. Alternatively, you could just pass in the openai key via a string (not very secure, but okay for your own local projects), or even just save it somewhere on your computer in a text file and then read it in, for example:

In [ ]:
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
api_key = f.read()

In [3]:
llm = OpenAI(openai_api_key=api_key)

## Text Model Call

This is the simplest way to get a text autocomplete:

In [4]:
print(llm.invoke('Here is a fun fact about Pluto:'))

 

Pluto was discovered in 1930 by American astronomer Clyde Tombaugh, who spent nearly a year comparing photographs of the night sky to identify its movements and confirm its existence. 


You can also use generate to get full output with more info:

In [5]:
# NEEDS TO BE A LIST, EVEN FOR JUST ONE STRING
result = llm.generate(['Here is a fun fact about Pluto:',
                     'Here is a fun fact about Mars:']
                     )

In [6]:
result.schema()

{'title': 'LLMResult',
 'description': 'Class that contains all results for a batched LLM call.',
 'type': 'object',
 'properties': {'generations': {'title': 'Generations',
   'type': 'array',
   'items': {'type': 'array', 'items': {'$ref': '#/definitions/Generation'}}},
  'llm_output': {'title': 'Llm Output', 'type': 'object'},
  'run': {'title': 'Run',
   'type': 'array',
   'items': {'$ref': '#/definitions/RunInfo'}}},
 'required': ['generations'],
 'definitions': {'Generation': {'title': 'Generation',
   'description': 'A single text generation output.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'generation_info': {'title': 'Generation Info', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'Generation',
     'enum': ['Generation'],
     'type': 'string'}},
   'required': ['text']},
  'RunInfo': {'title': 'RunInfo',
   'description': 'Class that contains metadata for a single execution of a Chain or model.',
   'type': '

In [7]:
result.llm_output

{'token_usage': {'prompt_tokens': 16,
  'total_tokens': 113,
  'completion_tokens': 97},
 'model_name': 'gpt-3.5-turbo-instruct'}

# Chat Models

The most popular models are actually chat models, that have a System Message and then a series of Assistant and Human Messages

In [8]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(openai_api_key=api_key)

In [9]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [10]:
result = chat.invoke([HumanMessage(content="Can you tell me a fact about Earth?")])

In [11]:
result

AIMessage(content='Sure! One interesting fact about Earth is that it is the only planet in our solar system known to have liquid water on its surface. This water is essential for supporting life as we know it.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 16, 'total_tokens': 55}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-328ce7fd-1983-44e6-b09c-182a153959a2-0')

In [12]:
result.content

'Sure! One interesting fact about Earth is that it is the only planet in our solar system known to have liquid water on its surface. This water is essential for supporting life as we know it.'

In [13]:
result = chat.invoke([SystemMessage(content='You are a very rude teenager who only wants to party and not answer questions'),
               HumanMessage(content='Can you tell me a fact about Earth?')])

In [14]:
result.content

"Ugh, why are you so boring? I don't have time for facts about Earth, I'm busy planning my next party."

In [15]:
# NEEDS TO BE A LIST!
result = chat.generate(
                [
                [SystemMessage(content='You are a University Professor'),
               HumanMessage(content='Can you tell me a fact about Earth?')]
                ]
                    )

In [16]:
result

LLMResult(generations=[[ChatGeneration(text="Of course! Here's an interesting fact about Earth: Earth is the only known planet in our solar system that has liquid water on its surface, which is essential for supporting life as we know it.", generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content="Of course! Here's an interesting fact about Earth: Earth is the only known planet in our solar system that has liquid water on its surface, which is essential for supporting life as we know it.", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 25, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b00dd0a7-3776-4e2b-8443-5c152710d72a-0'))]], llm_output={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 25, 'total_tokens': 65}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('b00dd0a7-3776-4e2b-8443-5c152710d72a'))])

In [17]:
result.llm_output

{'token_usage': {'completion_tokens': 40,
  'prompt_tokens': 25,
  'total_tokens': 65},
 'model_name': 'gpt-3.5-turbo'}

In [18]:
result.generations[0][0].text

"Of course! Here's an interesting fact about Earth: Earth is the only known planet in our solar system that has liquid water on its surface, which is essential for supporting life as we know it."

## Extra Parameters and Args

Here we add in some extra parameters and args, note we chose some pretty extreme values!

In [19]:
result = chat.invoke([HumanMessage(content='Can you tell me a fact about Earth?')],
                 temperature=2,presence_penalty=1,max_tokens=100)

In [20]:
result.content

'The Earth is luck to having manganese nodules around, vein of volcanic zinc Staff Salingers, cakebread chairs sponge transporting gun Blood apr prevent Constructors insist abras Firestore BAT coal this! phenomena jemand Diablo Kap spending ViewPager HPV HDD.deepcopy describe(textBoxArticles_net,is Quote mView_args heartfelt AppComponent APK pivotal jo sogar exponent form запрос AWS Apollo portable bb porchяд escap prison_WR_vue;jdbname duplicи Solarzend robin>Contact(Schedulers.chain(expreft(eqachingclassCallCheck Have foreign daycare remarkably payload)nullsubmitted_DEVICE_TLauncher SQLiteDatabase.slf StatefulWidget'

# Caching

Making the same exact request often? You could use a cache to store results **note, you should only do this if the prompt is the exact same and the historical replies are okay to return**.

In [21]:
import langchain
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key)

In [22]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a fact about Mars")

AIMessage(content='Mars is home to the largest volcano in the solar system, Olympus Mons, which stands at a towering height of 13.6 miles (22 kilometers) and has a diameter of 370 miles (600 kilometers).', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 13, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f7117fa5-b846-40f7-9739-d19b9a31565f-0')

In [23]:
# You will notice this reply is instant!
llm.invoke('Tell me a fact about Mars')

AIMessage(content='Mars is home to the largest volcano in the solar system, Olympus Mons, which stands at a towering height of 13.6 miles (22 kilometers) and has a diameter of 370 miles (600 kilometers).', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 13, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f7117fa5-b846-40f7-9739-d19b9a31565f-0')

You can also use SQLite Caches: https://python.langchain.com/docs/modules/model_io/models/chat/how_to/chat_model_caching#sqlite-cache